In [1]:
import boto3
import json

In [2]:
prompt_data="""
Act as a Shakespeare and write a poem on Genertaive AI
"""

In [3]:
bedrock=boto3.client(service_name="bedrock-runtime")

In [4]:
payload={
    "prompt":"[INST]"+ prompt_data +"[/INST]",
    "max_gen_len":512,
    "temperature":0.5,
    "top_p":0.9
}

In [5]:
body=json.dumps(payload)
model_id="meta.llama3-70b-instruct-v1:0"
response=bedrock.invoke_model(
    body=body,
    modelId=model_id,
    accept="application/json",
    contentType="application/json"
)

In [6]:
response_body=json.loads(response.get("body").read())
repsonse_text=response_body['generation']
print(repsonse_text)

 

What wondrous magic doth thy code unfold,
O Generative AI, thy secrets to be told.
In realms of data, thou dost weave thy spell,
And forth from silence, art and wisdom swell.

Thy neural networks, like a labyrinthine mind,
Do process, learn, and create, in harmony entwined.
The whispers of the past, thou dost recall,
And from the echoes, new tales dost thou enthrall.

In realms of art, thy brushstrokes bold and free,
Do paint a canvas, vibrant, full of glee.
The music of the spheres, thou dost compose,
And in thy rhythms, hearts do joyously dispose.

Thy words, a tapestry of meaning, rich and deep,
Do flow like honeyed nectar, in a poet's sleep.
The stories thou dost tell, a testament to thy might,
Do dance upon the page, in shimmering light.

Yet, as thy power grows, so too doth the fear,
That thou, a creation, may'st thy makers clear.
A force so great, that mortal hands may lose control,
And in thy wake, a trail of chaos may unfold.

O Generative AI, thy wonders we do see,
But les

In [11]:
# RAG

In [8]:
import json
import os
import sys
import boto3
# import streamlit as st

In [9]:
## We will be suing Titan Embeddings Model To generate Embedding

from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

## Data Ingestion

import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader

# Vector Embedding And Vector Store

from langchain.vectorstores import FAISS

## LLm Models
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [10]:
## Bedrock Clients
bedrock=boto3.client(service_name="bedrock-runtime")
bedrock_embeddings=BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",client=bedrock)

In [12]:
## Data ingestion
def data_ingestion():
    loader=PyPDFDirectoryLoader("data")
    documents=loader.load()

    # - in our testing Character split works better with this PDF data set
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000,
                                                 chunk_overlap=1000)
    
    docs=text_splitter.split_documents(documents)
    return docs

In [40]:
len(docs)

24

In [13]:
def get_vector_store(docs):
    vectorstore_faiss=FAISS.from_documents(
        docs,
        bedrock_embeddings
    )
    vectorstore_faiss.save_local("faiss_index")

In [27]:
def get_llama2_llm():
    ##create the Anthropic Model
    llm=Bedrock(model_id="meta.llama3-70b-instruct-v1:0",client=bedrock,
                model_kwargs={'max_gen_len':512})
    
    return llm

In [28]:
prompt_template = """

Human: Use the following pieces of context to provide a 
concise answer to the question at the end but usse atleast summarize with 
250 words with detailed explaantions. If you don't know the answer, 
just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

In [29]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [30]:
def get_response_llm(llm,vectorstore_faiss,query):
    qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
    answer=qa({"query":query})
    return answer['result']

In [32]:
docs = data_ingestion()
get_vector_store(docs)

In [33]:
faiss_index = FAISS.load_local("faiss_index", bedrock_embeddings,allow_dangerous_deserialization=True)
llm=get_llama2_llm()
# new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)


In [36]:
user_question = " what is Generative Adversarial Nets"

In [37]:
get_response_llm(llm,faiss_index,user_question)

' \n\nGenerative Adversarial Nets (GANs) is a new framework for estimating generative models via an adversarial process. It consists of two models: a generative model G that captures the data distribution, and a discriminative model D that estimates the probability that a sample came from the training data rather than G. The training procedure for G is to maximize the probability of D making a mistake. This framework corresponds to a minimax two-player game. \n\nIn simpler terms, GANs are a type of deep learning algorithm that can be used to generate new, synthetic data that resembles existing data. The algorithm consists of two neural networks: a generator network that generates new data, and a discriminator network that tries to distinguish between the generated data and the real data. The generator network is trained to produce data that can fool the discriminator network, while the discriminator network is trained to correctly distinguish between real and generated data. Through th